In [1]:

import numpy as np
import pandas as pd
import os
import cv2
import mediapipe as mp
import csv
import tqdm


In [2]:
# Initialize MediaPipe Pose solution
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Function to detect keypoints from an image using MediaPipe Pose
def detect(image):
    # Convert BGR image to RGB for MediaPipe processing
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Check if pose landmarks were detected
    if results.pose_landmarks:
        # Extract x, y, z coordinates and visibility scores for each landmark
        keypoints = [
            (lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark
        ]
        return keypoints
    return None  # Return None if no keypoints were found

In [3]:
# Class to handle preprocessing of pose images and saving data as CSV
class Preprocessor:
    def __init__(self, images_in_folder, csvs_out_path):
        self._images_in_folder = images_in_folder  # Folder containing images
        self._csvs_out_path = csvs_out_path  # Output CSV file path
        self._csvs_out_folder_per_class = "csv_per_pose"  # Folder to store class-wise CSVs
        self._messages = []  # List to store skipped image messages

        # Create the output folder if it does not exist
        if not os.path.exists(self._csvs_out_folder_per_class):
            os.makedirs(self._csvs_out_folder_per_class)

        # Get list of pose classes (subfolders inside images_in_folder)
        self._pose_class_names = sorted([
            name for name in os.listdir(images_in_folder)
            if os.path.isdir(os.path.join(images_in_folder, name))
        ])

    def process(self, detection_threshold=0.1):
        """
        Process all images, extract pose landmarks, and save results as CSV.
        """
        for pose_class_name in self._pose_class_names:
            images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
            csv_out_path = os.path.join(self._csvs_out_folder_per_class, pose_class_name + ".csv")

            with open(csv_out_path, "w", newline='') as csv_out_file:
                csv_out_writer = csv.writer(csv_out_file, delimiter=",", quoting=csv.QUOTE_MINIMAL)

                # Get list of image files (only png, jpg, jpeg)
                image_names = sorted([
                    name for name in os.listdir(images_in_folder)
                    if name.lower().endswith(('.png', '.jpg', '.jpeg'))
                ])
                valid_image_count = 0  # Counter for successfully processed images

                for image_name in tqdm.tqdm(image_names, desc=f"Processing {pose_class_name}"):
                    image_path = os.path.join(images_in_folder, image_name)
                    image = cv2.imread(image_path)  # Read the image

                    if image is None:
                        self._messages.append(f"Skipped {image_path}: Invalid image file")
                        continue
                    
                    # Ensure the image has 3 color channels (RGB)
                    if len(image.shape) < 3 or image.shape[2] != 3:
                        self._messages.append(f"Skipped {image_path}: Image is not in RGB format")
                        continue

                    # Detect pose keypoints
                    keypoints = detect(image)

                    if keypoints is None:
                        self._messages.append(f"Skipped {image_path}: No keypoints detected")
                        continue

                    # Check if minimum visibility score is above threshold
                    min_landmark_score = min(kp[3] for kp in keypoints)
                    if min_landmark_score < detection_threshold:
                        self._messages.append(f"Skipped {image_path}: Keypoints score below threshold")
                        continue

                    valid_image_count += 1  # Increment valid image count

                    # Extract x, y, and visibility (score) values from detected keypoints
                    pose_landmarks = np.array([[kp[0], kp[1], kp[3]] for kp in keypoints], dtype=np.float32)
                    # Write keypoints data to CSV
                    csv_out_writer.writerow([image_name] + pose_landmarks.flatten().astype(str).tolist())

        print("\n".join(self._messages))  # Print skipped image messages

        # Combine all class CSVs into one final CSV file
        all_landmarks_df = self.all_landmarks_as_dataframe()
        all_landmarks_df.to_csv(self._csvs_out_path, index=False)

    def class_names(self):
        """Returns the list of class names (pose categories)."""
        return self._pose_class_names

    def all_landmarks_as_dataframe(self):
        """
        Combine all individual class CSVs into a single DataFrame.
        """
        total_df = None
        for class_index, class_name in enumerate(self._pose_class_names):
            csv_out_path = os.path.join(self._csvs_out_folder_per_class, class_name + ".csv")
            if not os.path.exists(csv_out_path):
                continue
            
            per_class_df = pd.read_csv(csv_out_path, header=None)

            # Add numeric and string class labels
            per_class_df["class_no"] = class_index
            per_class_df["class_name"] = class_name

            # Append class name to filename in first column for better identification
            per_class_df.iloc[:, 0] = class_name + "/" + per_class_df.iloc[:, 0]

            if total_df is None:
                total_df = per_class_df
            else:
                total_df = pd.concat([total_df, per_class_df], axis=0)

        # Define column names: filename + (landmarks) + class labels
        header_names = ["filename"] + [f"{landmark.name}_{axis}" for landmark in mp_pose.PoseLandmark for axis in ["x", "y", "score"]]
        header_names += ["class_no", "class_name"]
        total_df.columns = header_names

        return total_df

In [4]:
# Paths for dataset processing
train_images_folder = r"C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train"
train_csv_out_path = "train_data.csv"

test_images_folder = r"C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test"
test_csv_out_path = "test_data.csv"

# Process training data
print("Processing Training Dataset...")
train_preprocessor = Preprocessor(train_images_folder, train_csv_out_path)
train_preprocessor.process()

# Process testing data
print("Processing Testing Dataset...")
test_preprocessor = Preprocessor(test_images_folder, test_csv_out_path)
test_preprocessor.process()

Processing Training Dataset...


Processing Warrior2: 100%|██████████| 822/822 [00:48<00:00, 17.01it/s]


Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\1047_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\10632_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\11082_aug.jpg: Keypoints score below threshold
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\11424.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\1183_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\11935_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\12046.jpg: Keypoints score below threshold
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\train\Downdog\12975.jpg: 

Processing Warrior2: 100%|██████████| 204/204 [00:11<00:00, 18.51it/s]


Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\11280.jpg: Keypoints score below threshold
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\11494.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\12545_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\14725_aug.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\16582.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\16829.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\17563.jpg: No keypoints detected
Skipped C:\Users\yenul\OneDrive\Desktop\DSGP project yoga\Processed_Dataset\test\Downdog\17578.jpg: No keypoints detected
Skippe